<a href="https://colab.research.google.com/github/tuenv193176/DATN/blob/main/DATN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chuẩn bị dữ liệu

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import numpy as np

# Đọc dữ liệu từ file
file_path = '/content/resonant_frequencies.csv'
df = pd.read_csv(file_path)

# Xử lý dữ liệu
X = df[['ls1 (mm)', 'ls2 (mm)', 'ls3 (mm)', 'ws1 (mm)', 'ws2 (mm)', 'ws3 (mm)']]
y = df[['Frequency 1 (GHz)', 'Frequency 2 (GHz)', 'Frequency 3 (GHz)']]

# Thêm cột chỉ báo số lượng tần số cộng hưởng
y = y.copy()
y.loc[:, 'num_frequencies'] = y.notna().sum(axis=1)

# Điền giá trị trung bình cho các giá trị thiếu (NaN)
y = y.fillna(y.mean())

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Chuẩn hóa tần số cộng hưởng
y_scaler = MinMaxScaler()
y_train_frequencies = y_train[['Frequency 1 (GHz)', 'Frequency 2 (GHz)', 'Frequency 3 (GHz)']]
y_train_frequencies = y_scaler.fit_transform(y_train_frequencies)
y_test_frequencies = y_scaler.transform(y_test[['Frequency 1 (GHz)', 'Frequency 2 (GHz)', 'Frequency 3 (GHz)']])

# Chuyển đổi dữ liệu sang tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train_frequencies = torch.tensor(y_train_frequencies, dtype=torch.float32)
y_test_frequencies = torch.tensor(y_test_frequencies, dtype=torch.float32)
y_train_num = torch.tensor(y_train['num_frequencies'].values, dtype=torch.float32).view(-1, 1)
y_test_num = torch.tensor(y_test['num_frequencies'].values, dtype=torch.float32).view(-1, 1)

# Tạo DataLoader
train_dataset = TensorDataset(X_train, y_train_frequencies, y_train_num)
test_dataset = TensorDataset(X_test, y_test_frequencies, y_test_num)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Định nghĩa và Huấn luyện Mô hình

In [2]:
# Định nghĩa mô hình ANN
class ANN_Model(nn.Module):
    def __init__(self):
        super(ANN_Model, self).__init__()
        self.fc1 = nn.Linear(6, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5_freq = nn.Linear(32, 3)  # Dự đoán tần số cộng hưởng
        self.fc5_num = nn.Linear(32, 1)  # Dự đoán số lượng tần số cộng hưởng
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout(x)
        x = torch.relu(self.fc4(x))
        frequencies = self.fc5_freq(x)
        num_frequencies = torch.sigmoid(self.fc5_num(x)) * 3  # Đầu ra là giữa 0 và 3
        return frequencies, num_frequencies

# Khởi tạo mô hình, định nghĩa hàm mất mát và bộ tối ưu
model = ANN_Model()
criterion_freq = nn.MSELoss()
criterion_num = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Huấn luyện mô hình
epochs = 1000
for epoch in range(epochs):
    model.train()
    running_loss_freq = 0.0
    running_loss_num = 0.0
    for inputs, targets_freq, targets_num in train_loader:
        optimizer.zero_grad()
        outputs_freq, outputs_num = model(inputs)
        loss_freq = criterion_freq(outputs_freq, targets_freq)
        loss_num = criterion_num(outputs_num, targets_num)
        loss = loss_freq + loss_num
        loss.backward()
        optimizer.step()
        running_loss_freq += loss_freq.item()
        running_loss_num += loss_num.item()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss Frequency: {running_loss_freq/len(train_loader):.4f}, Loss Num: {running_loss_num/len(train_loader):.4f}')


Epoch [100/1000], Loss Frequency: 0.0085, Loss Num: 0.0431
Epoch [200/1000], Loss Frequency: 0.0074, Loss Num: 0.0228
Epoch [300/1000], Loss Frequency: 0.0067, Loss Num: 0.0145
Epoch [400/1000], Loss Frequency: 0.0059, Loss Num: 0.0116
Epoch [500/1000], Loss Frequency: 0.0066, Loss Num: 0.0093
Epoch [600/1000], Loss Frequency: 0.0061, Loss Num: 0.0093
Epoch [700/1000], Loss Frequency: 0.0055, Loss Num: 0.0096
Epoch [800/1000], Loss Frequency: 0.0046, Loss Num: 0.0089
Epoch [900/1000], Loss Frequency: 0.0051, Loss Num: 0.0080
Epoch [1000/1000], Loss Frequency: 0.0057, Loss Num: 0.0144


# Kiểm tra Mô hình

In [3]:
# Kiểm tra mô hình
def evaluate_model(model, test_loader, criterion_freq, criterion_num):
    model.eval()
    test_loss_freq = 0.0
    test_loss_num = 0.0
    with torch.no_grad():
        for inputs, targets_freq, targets_num in test_loader:
            outputs_freq, outputs_num = model(inputs)
            loss_freq = criterion_freq(outputs_freq, targets_freq)
            loss_num = criterion_num(outputs_num, targets_num)
            test_loss_freq += loss_freq.item()
            test_loss_num += loss_num.item()

    avg_test_loss_freq = test_loss_freq / len(test_loader)
    avg_test_loss_num = test_loss_num / len(test_loader)
    print(f'Test Loss Frequency: {avg_test_loss_freq:.4f}, Test Loss Num: {avg_test_loss_num:.4f}')

evaluate_model(model, test_loader, criterion_freq, criterion_num)


Test Loss Frequency: 0.0093, Test Loss Num: 0.1256


In [4]:
# Dự đoán với 5 hàng đầu trong tập dữ liệu
def predict_top_n(model, X_data, scaler, y_scaler, n=53):
    model.eval()
    input_df = pd.DataFrame(X_data[:n], columns=['ls1 (mm)', 'ls2 (mm)', 'ls3 (mm)', 'ws1 (mm)', 'ws2 (mm)', 'ws3 (mm)'])
    input_data_scaled = scaler.transform(input_df)  # Chuẩn hóa dữ liệu
    input_tensor = torch.tensor(input_data_scaled, dtype=torch.float32)
    with torch.no_grad():
        pred_freq, pred_num = model(input_tensor)
    pred_freq = y_scaler.inverse_transform(pred_freq.numpy())  # Chuyển đổi ngược lại tần số cộng hưởng
    pred_num = pred_num.numpy()
    # Kiểm tra và loại bỏ tần số cộng hưởng thứ ba nếu số lượng dự đoán là 2
    for i in range(len(pred_num)):
        if pred_num[i] < 2.5:  # Nếu dự đoán số lượng tần số cộng hưởng nhỏ hơn 2.5, coi như có 2 tần số
            pred_freq[i][2] = np.nan
    return pred_freq

# Lấy 5 hàng đầu tiên từ dữ liệu gốc để kiểm tra
top_n_input = df[['ls1 (mm)', 'ls2 (mm)', 'ls3 (mm)', 'ws1 (mm)', 'ws2 (mm)', 'ws3 (mm)']].head(5).values
top_n_actual = df[['Frequency 1 (GHz)', 'Frequency 2 (GHz)', 'Frequency 3 (GHz)']].head(5).values

# Dự đoán
predicted_frequencies_top_n = predict_top_n(model, top_n_input, scaler, y_scaler, n=5)

# Hiển thị kết quả
print("Actual Frequencies:\n", top_n_actual)
print("Predicted Frequencies:\n", predicted_frequencies_top_n)


Actual Frequencies:
 [[2.55 5.3  5.85]
 [2.5  5.45  nan]
 [2.5  5.45  nan]
 [2.5  5.35  nan]
 [2.5  5.3  5.85]]
Predicted Frequencies:
 [[2.5171857 5.249425  5.8012443]
 [2.4835267 5.3392286       nan]
 [2.463023  5.376052        nan]
 [2.4989805 5.307441        nan]
 [2.4903371 5.2584786 5.815308 ]]


In [15]:
# Dự đoán với input cụ thể
def predict(model, input_data, scaler, y_scaler):
    model.eval()
    # Chuyển đổi input thành DataFrame tạm thời
    input_df = pd.DataFrame([input_data], columns=['ls1 (mm)', 'ls2 (mm)', 'ls3 (mm)', 'ws1 (mm)', 'ws2 (mm)', 'ws3 (mm)'])
    input_data_scaled = scaler.transform(input_df)  # Chuẩn hóa dữ liệu
    input_tensor = torch.tensor(input_data_scaled, dtype=torch.float32)
    with torch.no_grad():
        pred_freq, pred_num = model(input_tensor)
    pred_freq = y_scaler.inverse_transform(pred_freq.numpy())  # Chuyển đổi ngược lại tần số cộng hưởng
    pred_num = pred_num.numpy()
    # Kiểm tra và loại bỏ tần số cộng hưởng thứ ba nếu số lượng dự đoán là 2
    if pred_num < 2.5:  # Nếu dự đoán số lượng tần số cộng hưởng nhỏ hơn 2.5, coi như có 2 tần số
        pred_freq[0][2] = np.nan
    return pred_freq

In [16]:
# Ví dụ input mới
new_input = [7.6, 9.1, 7.8, 0.7, 1.85, 0.3]
predicted_frequencies = predict(model, new_input, scaler, y_scaler)
print(f'Dự đoán tần số cộng hưởng: {predicted_frequencies}')
#Ket qua thuc te tren HFSS: 2.45 5.25 5.9

Dự đoán tần số cộng hưởng: [[2.4573414 5.2167425 5.875263 ]]
